**Prod Version of DevEnv**

**This notebook is run using nbconvert after devenv service starts**

In [ ]:
!pip install twitter
!pip install feedparser
!pip install schedule
!pip install kafka-python
!pip install redis
!pip install uuid

In [ ]:
from time import sleep
from json import dumps, loads, load
from kafka import KafkaProducer
from datetime import datetime, timedelta
from feedparser import parse
from urllib.request import urlopen
from twitter import Twitter
import schedule
from datetime import datetime
import uuid

In [ ]:
tickers = ["MSFT","AAPL"]
feeds = ["http://feeds.reuters.com/reuters/businessNews"]

producer = KafkaProducer(
    bootstrap_servers=['broker:9092'],
    api_version= (0,11),
    value_serializer=lambda x: dumps(x).encode('utf-8'))

# Authenticate with Twitter & search
# https://pypi.org/project/twitter/
# https://developer.twitter.com/en/apps/17019807
from twitter import *
t = Twitter(auth=OAuth(consumer_key="nDwKHutvaskyGecZkCS1SiYY8", consumer_secret="zlQdJIuFUGGglbMO2kYMk6oNLFztzipxhRANNz5q5gqfn1tGb1", token="73568030-GqfoqkfWRhqhARS9uzz51vg1UGSV7AGRaOxDkAurZ", token_secret="1kiHGfyDfO7gcaJbGFnr3lnLA3vXc8ti9hJhj1lnTTvEm"))

In [ ]:
def DownloadStreams():
    # Yahoo Finance Stream for Stock Price
    f_data = {}
    for ticker in tickers:
        try:
            y_finance = load(urlopen("https://query1.finance.yahoo.com/v7/finance/chart/"+ticker+"?range=1h&interval=1h"))
            meta = y_finance['chart']['result'][0]['meta']
            timestamp = meta['regularMarketTime']
            price = meta['regularMarketPrice']
            f_data[ticker] = {"Type": "Finance", "Ticker": ticker, "Timestamp":timestamp, "Price":price}
        except:
            print("Exception in Download Stream for "+ticker)
    
    # News Stream from Reuters for Business
    feed_data = {}
    for feed in feeds:
        feed_data[feed] = [{"Type": "News", "Timestamp": datetime.timestamp(datetime.now()), "Title":l.title, "Summary":l.summary, "Link":l.link} for l in (parse(feed))["entries"]]
        # print([l.title for l in d["entries"]])

    return f_data, feed_data

def UploadStreamsToKafka(f_data, feed_data, PopularityOnly=False):
    if(not PopularityOnly):
        for key in f_data.keys():
            producer.send('finance_ticker', value=f_data[key])
        for key in feed_data.keys():
            for entry in feed_data[key]:
                producer.send('news_feed', value=entry)
    # Create Popularity Click Feed
    for key in feed_data.keys():
        for entry in feed_data[key]:
            a = t.search.tweets(q=entry["Title"])
            for s in a['statuses']:
                if(s['retweet_count']>0):
                    producer.send('click',value={'Type': 'Click', 'User': 'generic_'+str(uuid.uuid4()), 'Timestamp': datetime.timestamp(datetime.now()), 'Title': entry["Title"], 'Link': entry["Link"] })
    producer.flush()

def HourlyJob():
    # Check, Download & Parse each stream
    f_data, feed_data = DownloadStreams()
    # Upload cleaned events into Kafka Topics
    UploadStreamsToKafka(f_data, feed_data)
    print("Hourly Run Events Complete - Sleeping now")

def MinutelyJob():
    f_data, feed_data = DownloadStreams()
    UploadStreamsToKafka(f_data, feed_data, True)
    print("Minutely Run Events Complete - Sleeping now")



In [ ]:
HourlyJob()

In [ ]:
MinutelyJob()

In [ ]:
schedule.clear()
HourlyJob()

schedule.every(15).minutes.do(HourlyJob)
schedule.every(1).minutes.do(MinutelyJob)
starttime = datetime.now()
endtime = starttime + timedelta(minutes=5)

while True: #datetime.now()<endtime:
    schedule.run_pending()
    sleep(10)
    
schedule.clear()